In [22]:
from implementations import *
from data_cleanup import *
from functools import partial
np.warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
DATA_TRAIN_PATH = 'data/train.csv' 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

Test data cleaning functions

In [4]:
xs, ys = preprocess_train(tX, y)

x_0 shape: (99913, 30), y_0 shape: (99913,)
x_1 shape: (77544, 30), y_1 shape: (77544,)
x_2 shape: (50379, 30), y_2 shape: (50379,)
x_3 shape: (22164, 30), y_3 shape: (22164,)


In [78]:
initial_w = np.zeros(30)
gamma = 0.03
max_iters = 2
batch_size = 2000
ratio = 0.9

Test logistic regression using Newton method

In [79]:
weights = []
loss = []
for i in range(4):
    print(f'***{i}***')
    w , l = logistic_regression_newton_method(ys[i], xs[i], initial_w, max_iters, batch_size, ratio, gamma)
    print(f'remove misclassified points')
    dist = np.dot(xs[i], w)
    y_dist = dist*[-1 if x==0 else 1 for x in list(ys[i])]
    val = np.percentile(y_dist, 5)
    x_new = xs[i][(~(y_dist < val))]
    y_new = ys[i][(~(y_dist < val))]
    print(f'new model from {xs[i].shape} to {x_new.shape} data')
    w , l = logistic_regression_newton_method(y_new, x_new, w, max_iters, batch_size, ratio, gamma)
    print(f'remove misclassified points')
    dist = np.dot(x_new, w)
    y_dist = dist*[-1 if x==0 else 1 for x in list(y_new)]
    val = np.percentile(y_dist, 5)
    x_new2 = x_new[(~(y_dist < val))]
    y_new2 = y_new[(~(y_dist < val))]
    print(f'new model from {x_new.shape} to {x_new2.shape} data')
    w , l = logistic_regression_newton_method(y_new2, x_new2, w, max_iters, batch_size, ratio, gamma)
    weights.append(w)
    loss.append(l)

***0***
epoch: 0, train_loss: 28848.744422494485, test_loss: 2413.139727142242, test accuracy: 0.8203562850280224
epoch: 1, train_loss: 18127.884861085295, test_loss: 1884.2057331557767, test accuracy: 0.8221577261809447
remove misclassified points
new model from (99913, 30) to (94917, 30) data
epoch: 0, train_loss: 12965.583576113604, test_loss: 1316.7764236521307, test accuracy: 0.8656763590391909
epoch: 1, train_loss: 11163.120400298743, test_loss: 1217.8886576970735, test accuracy: 0.8660977665402444
remove misclassified points
new model from (94917, 30) to (90171, 30) data
epoch: 0, train_loss: 7451.636464561068, test_loss: 794.7547939418033, test accuracy: 0.9062985140829453
epoch: 1, train_loss: 6737.506365530297, test_loss: 746.0743989642757, test accuracy: 0.9066311820802839
***1***
epoch: 0, train_loss: 23837.575684720086, test_loss: 2370.2830802574936, test accuracy: 0.716441005802708
epoch: 1, train_loss: 19027.428658622615, test_loss: 2086.552991978488, test accuracy: 0.71

In [85]:
OUTPUT_PATH = 'data/test.csv' 
_, tX_test, ids_test = load_csv_data(OUTPUT_PATH)

In [106]:
tx_test, idx = preprocess_test(tX_test, ids_test)

x_0 shape: (227458, 30)
x_1 shape: (175338, 30)
x_2 shape: (114648, 30)
x_3 shape: (50794, 30)


In [132]:
labels = []
for i in range(4):
    labels.append(predict_logistic_labels(weights[i], tx_test[i]))
final = np.array(list(labels[0])+list(labels[1])+list(labels[2])+list(labels[3]))
final = final * 2 - 1
final_idx = np.array(list(idx[0])+list(idx[1])+list(idx[2])+list(idx[3]))
create_csv_submission(final_idx, final, 'bkm4.csv')